In [1]:
import sys
REPO_PATH='/home/brook/kaggle/planet/repo'
sys.path.append(REPO_PATH)

In [2]:
import os
from importlib import reload
import models.vgglike as vl
import generators.csvgen as cg
import utils
reload(vl)
reload(utils)
reload(cg)

Using TensorFlow backend.


<module 'generators.csvgen' from '/home/brook/kaggle/planet/repo/generators/csvgen.py'>

In [3]:
# USE THESE SETTINGS FOR A REALISTIC SAMPLE
KG_ROOT='/home/brook/kaggle/planet'
DATA_ROOT=os.environ.get('DATA')
DATA_DIR='planet'
WEIGHT_ROOT=os.environ.get('WEIGHTS')
IMG_TYPE='tif'

In [4]:
ROOT=f'{DATA_ROOT}/{DATA_DIR}'
WEIGHT_DIR=f'{WEIGHT_ROOT}/{DATA_DIR}'
JPG_DIR = os.path.join(ROOT, 'train-jpg')
TIF_DIR = os.path.join(ROOT, 'train-tif')

In [5]:
if IMG_TYPE=='tif':
    BANDS=4
    IMG_DIR=TIF_DIR
else:
    BANDS=3
    IMG_DIR=JPG_DIR

In [6]:
TRAIN_SIZE=200
VALID_SIZE=50
BATCH_SIZE=32
CSV_DIR=f'{KG_ROOT}/play'
TRAIN_CSV=f'{CSV_DIR}/train-sample.csv'
VALID_CSV=f'{CSV_DIR}/valid-sample.csv'

In [7]:
train=cg.CSVGen(TRAIN_CSV,image_dir=IMG_DIR,batch_size=BATCH_SIZE)
valid=cg.CSVGen(VALID_CSV,image_dir=IMG_DIR,batch_size=BATCH_SIZE)

## VGG-ARCH

In [20]:
vggarch=vl.VGGARCH(batch_input_shape=(None,256, 256,BANDS))

In [21]:
vggarch.model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 256, 256, 4)       16        
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 258, 258, 4)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 256, 256, 32)      1184      
_________________________________________________________________
zero_padding2d_17 (ZeroPaddi (None, 258, 258, 32)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
zero_padding2d_18 (ZeroPaddi (None, 130, 130, 32)      0         
__________

In [22]:
vggarch.fit_gen(TRAIN_SIZE,VALID_SIZE,6,
             train_gen=train.data(),valid_gen=valid.data(),sample_pct=0.4)

Epoch 1/6
13/13 [==============================] - 233s - loss: 4.0874 - acc: 0.7130 - val_loss: 2.3148 - val_acc: 0.8555

In [31]:
VGGARCH_WEIGHTS='vggarch40.h5'
dvgg.model().save_weights(f'{WEIGHT_DIR}/{VGGARCH_WEIGHTS}')

## VGGARCH-2 (cos-distance)

In [8]:
vggarch2=vl.VGGARCH(batch_input_shape=(None,256, 256,BANDS),loss_func=utils.cos_distance)

In [12]:
vggarch2.fit_gen(TRAIN_SIZE,VALID_SIZE,6,
             train_gen=train.data(),valid_gen=valid.data())

Epoch 1/6
33/33 [==============================] - 554s - loss: 0.5255 - acc: 0.4371 - val_loss: 0.5526 - val_acc: 0.6709

In [14]:
VGGARCH_WEIGHTS='vggarch_cos.h5'
vggarch2.model().save_weights(f'{WEIGHT_DIR}/{VGGARCH_WEIGHTS}')

## DUMMY

In [23]:
dvgg=vl.DummyVGG(batch_input_shape=(None,256, 256,BANDS))

In [33]:
dvgg.model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 256, 256, 4)       16        
_________________________________________________________________
zero_padding2d_31 (ZeroPaddi (None, 258, 258, 4)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 256, 256, 32)      1184      
_________________________________________________________________
zero_padding2d_32 (ZeroPaddi (None, 258, 258, 32)      0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 524288)            0         
__________

In [25]:
dvgg.fit_gen(TRAIN_SIZE,VALID_SIZE,6,
             train_gen=train.data(),valid_gen=valid.data(),sample_pct=0.4)

Epoch 1/6
13/13 [==============================] - 55s - loss: 1.5004 - acc: 0.8712 - val_loss: 1.6407 - val_acc: 0.8979

In [32]:
DUMMY_WEIGHTS='dummyvgg40.h5'
dvgg.model().save_weights(f'{WEIGHT_DIR}/{DUMMY_WEIGHTS}')

## DUMMY2 (cos-distance)

In [15]:
dvgg2=vl.DummyVGG(batch_input_shape=(None,256, 256,BANDS),loss_func=utils.cos_distance)

In [16]:
dvgg2.fit_gen(TRAIN_SIZE,VALID_SIZE,6,
             train_gen=train.data(),valid_gen=valid.data())

Epoch 1/6
33/33 [==============================] - 139s - loss: 0.4470 - acc: 0.2719 - val_loss: 0.4077 - val_acc: 0.3544

In [17]:
DUMMY_WEIGHTS='dummyvgg_cos.h5'
dvgg2.model().save_weights(f'{WEIGHT_DIR}/{DUMMY_WEIGHTS}')